导入依赖

In [2]:
import os,models
import loaders
import pytorch_lightning as pl
from torch import nn
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms
from utils.Functions import *
from argparse import ArgumentParser
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

/home/wuzhican/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wuzhican/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


配置参数

In [3]:
batch_size = 2
root_dir='./data'
data_save_root='./checkpoint'
style_image_path='./data/style.jpeg'
models_choice_from = [
    "ImfwNet",
    "SMNet"
]
arg_v = {
    'model':'ImfwNet',
    'learning_rate':1e-3,
    'root_dir':root_dir,
    'data_save_root':root_dir,
    'style_image_path':style_image_path,
    'batch_size':batch_size,
    'style_weight':10,
    'content_weight':1e3,
    'tv_weight':1e-3,
    'resume_path':data_save_root,
    'lr':1e-3,
}

model载入

In [4]:
%matplotlib inline
if arg_v['model'] not in models_choice_from:
    print('model choice is not in %s,exit'%(str(models_choice_from)))
    exit(-1)
else:
    if arg_v['model'] == 'ImfwNet':
        module = models.FWNetModule(
            load_image(style_image_path,shape=(512,512)),
            style_weight=arg_v['style_weight'],
            content_weight=arg_v['content_weight'],
            tv_weight=arg_v['tv_weight'],
            content_layers = ['layer2_2', 'layer3_3'],
            style_layers = ['layer2_2','layer4_3'],
            automatic_optimization=False,
            lr=arg_v['lr']
        )
        train_dataset = loaders.styleLoader(root_dir,augment_ratio=2)
        loader = (
            DataLoader(train_dataset, batch_size=batch_size,num_workers=2,drop_last=True),
        )
        hooks = [EarlyStopping(monitor='train_loss', min_delta=0.1, patience=3, verbose=False, mode="min")]
        logger = TensorBoardLogger("./data/lightning_logs", name='ImfwNet')
        models_args={
            'logger':logger
        }
    elif arg_v['model'] == 'SMNet':
        module = models.SMNet(
            load_image(style_image_path,shape=(512,512)),
            style_weight=arg_v['style_weight'],
            content_weight=arg_v['content_weight'],
            automatic_optimization=False,
            content_layers = ['layer2_2'],
            style_layers = ['layer3_3', 'layer4_3']
        )
        train_dataset = loaders.styleLoader(root_dir,augment_ratio=int(1e9))
        loader = (
            DataLoader(train_dataset, batch_size=batch_size,num_workers=2,drop_last=True),
        )
        hooks = [EarlyStopping(monitor='loss', min_delta=1e-2, patience=9, verbose=False, mode="min")]
        logger = TensorBoardLogger('./data/lightning_logs', name='SMNet')
        models_args={
            'logger':logger
        }

/home/wuzhican/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/wuzhican/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


: 

: 

In [ ]:
trainer = pl.Trainer(callbacks=hooks, default_root_dir=data_save_root,**models_args)
trainer.fit(module,*loader)